In [ ]:
from openai import OpenAI
import os
import json

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = client.responses.create(
    model="gpt-5",
    input="Hello, world!",
)

response.output_text

### Minimal Reasoning effort

In [ ]:
response = client.responses.create(
    model="gpt-5",
    input="Which number is larger 5.11 or 5.8",
    reasoning={"effort": "minimal"},
)

response.output_text

In [ ]:
response = client.responses.create(
    model="gpt-5",
    input="Which number is larger 5.11 or 5.8",
    reasoning={"effort": "high", "summary": "detailed"},
)

response.output

### Responses API for optimal performance

In [ ]:
response = client.chat.completions.create(
    model="gpt-5",
    messages=[
        {"role": "user", "content": "Hello, world!"},
    ],
)


In [ ]:
response.choices[0].message.content

In [ ]:
response = client.responses.create(
    model="gpt-5",
    input="Hello, world!",
)

In [ ]:
response.output_text

In [ ]:
response.output

In [ ]:
response = client.responses.create(
    model="gpt-5",
    input="Hello, world!",
    include=["reasoning.encrypted_content"],
    store=False,
)

In [ ]:
response

### When is the reasoning items necessary for model intelligence?
(user) -> user message -> (api) -> toolcall, reasoning item -> (user) --> toolcall result, reasoning item ->

(api) -> toocall2, reasoning item2 -> (user) -> toolcall result2, reasoning item2 -> (api) -> assistant message -> (user)

In [ ]:
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]

In [ ]:
context = [{"role": "user", "content": "What's the weather like in Paris today?"}]

response = client.responses.create(
    model="gpt-5",
    input=context,
    tools=tools,
    store=False, #store=false, just like how ZDR is enforced
    include=["reasoning.encrypted_content"] # Encrypted chain of thought is passed back in the response
)

In [ ]:
response

In [ ]:
context += response.output # Add the response to the context (including the encrypted chain of thought)
tool_call = response.output[1]
args = json.loads(tool_call.arguments)



result = 20 #mocking the result of the function call

context.append({                               
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)
})

response_2 = client.responses.create(
    model="gpt-5",
    input=context,
    tools=tools,
    store=False,
    include=["reasoning.encrypted_content"]
)

print(response_2.output_text)

## Verbosity and free form toolcalling

In [ ]:
response = client.responses.create(
    model="gpt-5",
    input="Hello, world!",
    text={"verbosity": "low"},
)

response.output_text

In [ ]:
response = client.responses.create(
    model="gpt-5",
    input="Hello, world!",
    text={"verbosity": "high"},
)

response.output_text

In [ ]:
def verbosity_test(verbosity: str, prompt: str, reasoning: str = "medium"):



    # tools = [
    #     {
    #         "type": "function",
    #         "name": "write_code",
    #         "description": "Tool used to output the code",
    #         "strict": True,  # ensure the model adheres to the JSON schema
    #         "parameters": {
    #             "type": "object",
    #             "properties": {
    #                 "code": {
    #                     "type": "string",
    #                     "description": "String for the code generated"
    #                 }
    #             },
    #             "required": ["code"],
    #             "additionalProperties": False
    #         }
    #     }
    # ]


    TOOLS = [
    {
        "type": "custom",
        "name": "write_code",
        "description": "tool used to output code",
    }]


    resp = client.responses.create(
    model="gpt-5",
    input=prompt,
    tools=TOOLS,
    tool_choice="auto",
    text={"verbosity": verbosity, "format": {"type": "text"}},
    reasoning={"effort": reasoning},
    
    )
    emitted_code = None
    # print(resp)

    # for item in getattr(resp, "output", []) or []:
    #     if getattr(item, "type", None) == "function_call" and getattr(item, "name", "") == "write_code":
    #         args = getattr(item, "arguments", {}) or {}
    #         if isinstance(args, str):
    #             try:
    #                 args = json.loads(args)
    #             except json.JSONDecodeError:
    #                 args = {}
    #         emitted_code = args.get("code")
    #         break

    emitted_code = resp.output[1].input




    print("emitted_code: \n", emitted_code)
    
    return emitted_code






In [ ]:
prompt = (
    "Write a Python function that computes factorial(n) iteratively and handles n=0. "
    "Return the result using the write_code tool, putting the code."
)


In [ ]:
code = verbosity_test("low",prompt)

In [ ]:
code = verbosity_test("high", prompt)


In [ ]:
prompt = """write me a python script that I can copy and paste into a jupyter notebook that will make a detailed drawing of a minecraft dirt block """


In [ ]:
code = verbosity_test("high", prompt, "high")
